<a href="https://colab.research.google.com/github/bryleen1/DfE_python/blob/main/TextAnalysisRegex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Text Analysis

## What kinds of text analysis are there?

* analyst knows the pattern
    * regular expressions
* analyst does not know the pattern
    * natural language processing
        * compares historical examples to judge novel cases
            * comparisons are statistical and approximate
            

### Examples of Analysis

When you know the pattern:

In [1]:
pattern = '£ ?[0-9][0-9]?' # £ then SPACE-optional then digit then digit-optional 

document = 'My eggs cost £3, bread cost £2, vodka cost £35'

In [2]:
import re

In [3]:
re.findall(pattern, document)

['£3', '£2', '£35']

If you dont:

* sentiment analysis
    * how positive/negative is this (new) review?
* topic analysis 
    * what is this document about?

## What can I do if I know what pattern I want to find?

* finding ("extracting")
    * what matches the pattern?
* matching ("validating")
    * does the entire document match YES/NO?
* substitue ("replacing")
    * replace a part that matches a pattern with another...

## How do I validate text with pandas?

In [6]:
import pandas as pd

In [7]:
ti = pd.read_csv('datasets/titanic.csv')
ti.sample(1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
505,0,1,male,18.0,1,0,108.9,C,First,man,True,C,Cherbourg,no,False


In [8]:
ti['ticket'] = "Ticket: " + ti['class'] + "; Price: $ " + ti['fare'].astype(str) + "; Port: " + ti['embark_town'] + ";"

In [9]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [10]:
pattern = '(First|Second)'

ti['class'].str.match(pattern)

0      False
1       True
2      False
3       True
4      False
       ...  
886     True
887     True
888    False
889     True
890    False
Name: class, Length: 891, dtype: bool

In [11]:
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.5575

## How do I extract data with pandas?

In [12]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [13]:
pattern = '([0-9.]+)'

ti['ticket'].str.extract(pattern).sample(4)

,0
67,8.1583
298,30.5
43,41.5792
639,16.1


## How do I substitue text with pandas?

In [14]:
ti['ticket'].str.replace('$', '€').sample(1)

881    Ticket: Third; Price: € 7.8958; Port: Southamp...
Name: ticket, dtype: object

## What are regular expressions?

Regular expressions are a language for describing patterns in text. 

They are separate from python, but may be used within python program. (And elsewhere, eg., often in SQL). 

They are notoriously difficult to read and write; and as a separate language, an additional tool to learn. 

## What regular expression patterns can I use?

* literals
    * `a`, find me an `a`
    * `£`, find `£`
    * `!` means `!` 
    * ... most symbols mean "find me"
* `.`
    * find any **single** symbol 
* character classes -- find a **single** symbol
    * `[abc]` $\rightarrow$ **either** a, b, c
    * `[0-9]` $\rightarrow$ **either** 0, 1, 2, 3,...9
    * `[A-Z]` $\rightarrow$ **either** capital A, B, ... Z
    * inversions
        * `[^abc]` $\rightarrow$ **is not** `a` OR `b` OR `c`
        * `[^a-zA-Z0-9 ]`  $\rightarrow$ **is not** alphanumeric-ish
    
* alternatives -- find the character**s** given by...
    * `(May|June|July)`  $\rightarrow$ **the whole worlds** May OR June..
    

In [25]:
ti['ticket'].str.extract('(Ticket: (First|Second))')

,0,1
0,NaN,NaN
1,Ticket: First,First
2,NaN,NaN
3,Ticket: First,First
4,NaN,NaN
...,...,...
886,Ticket: Second,Second
887,Ticket: First,First
888,NaN,NaN
889,Ticket: First,First


In [26]:
ti['ticket'].str.extract('( [0-9][0-9])')

,0
0,NaN
1,71
2,NaN
3,53
4,NaN
...,...
886,13
887,30
888,23
889,30


In [27]:
ti['ticket'].sample(1)

560    Ticket: Third; Price: $ 7.75; Port: Queenstown;
Name: ticket, dtype: object

In [28]:
ti['ticket'].str.extract('(Ticket: [A-Z])').sample(2)

,0
815,Ticket: F
516,Ticket: S


In [29]:
ti['ticket'].str.extract('(T........)').sample(3)

,0
186,Ticket: T
405,Ticket: S
290,Ticket: F


In [30]:
ti['ticket'].str.extract('(Price: [^0-9A-Za-z] ..)').sample(3)

,0
447,Price: $ 26
693,Price: $ 7.
423,Price: $ 14


In [31]:
ti['ticket'].str.extract('(Port: (Cherbourg|Southampton))').sample(3)

,0,1
57,Port: Cherbourg,Cherbourg
87,Port: Southampton,Southampton
655,Port: Southampton,Southampton


* repetitions
    * optional `?`
        * an optional number: `[0-9]?`
    * one or more `+`
        * one or more spaces: ` +`  
    * optional, or more, `*`
        * ` [0-9][0-9]?.[0-9]*`
    

In [32]:
ti['ticket'].str.extract('([0-9][0-9]?.[0-9]*)').sample(3)

,0
682,9.225
812,10.5
716,227


In [33]:
ti['ticket'].str.extract('(Ticket: [a-zA-Z]+)').sample(3)

,0
191,Ticket: Second
435,Ticket: First
49,Ticket: Third


In [34]:
row = 0
match = 1 # second match

ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+)').loc[row, match]

0    Port: Southampton
Name: (0, 1), dtype: object

In [35]:
ti['ticket'].str.extract('([a-zA-Z]+tow?n)')

,0
0,Southampton
1,NaN
2,Southampton
3,Southampton
4,Southampton
...,...
886,Southampton
887,Southampton
888,Southampton
889,NaN


* EXTRA: 
    * escaping
        * How do I say, literally, the `.` symbol?
        * `\.`
    

In [36]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

,0
269,$ 135.6333
685,$ 41.5792


* positional matching
    * `^` means **at the beginning**
    * `$` means **at the end**

In [37]:
ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+;$)').sample(1)

,,0
,match,
593,0,Port: Queenstown;


In [38]:
ti['ticket'].str.extractall('(^[a-zA-Z]+: [a-zA-Z]+;)').sample(1)

,,0
,match,
519,0,Ticket: Third;


## Next Steps

* review a "Regex Cheat Sheet"
    * also, eg., https://en.wikipedia.org/wiki/Regular_expression#Examples

## Exercise (30 min)

* find all the words in the tickets 
    * HINT: a word is a repeated letter followed by a space or a colon
    * HINT: `[ :]` means a space or a colon
* find all the USD prices
    * HINT: ``` \$ ``` and repeated numbers 
    
* find all the high-price tickets
    * HINT: consider `\$`, tripple-digit number, `\.`

In [ ]:
Solution

In [15]:
ti['ticket'].str.findall('([a-zA-Z]*[ :])').sample(10)

689    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
379    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
597    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
450    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
617    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
145    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
820    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
562    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
647    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
310    [Ticket:,  ,  , Price:,  ,  ,  , Port:,  ]
Name: ticket, dtype: object

In [16]:
ti['ticket'].str.extract('(\$ [0-9]+)').sample(2)

,0
233,$ 31
794,$ 7


In [24]:
row = 0

match = 1 # second match

#or

#ti['ticket'].str.extractall('(\$ [0-9][0-9][0-9]+\.[0-9]+)')#.loc[row, 0] # same thing

ti['ticket'].str.extractall('(\$ [0-9]{3})')#.loc[row, 0] # more efficient using {}   Exactly the specified number of occurrences

,,0
,match,
27,0,$ 263
31,0,$ 146
88,0,$ 263
118,0,$ 247
195,0,$ 146
215,0,$ 113
258,0,$ 512
268,0,$ 153
269,0,$ 135
